# Creating random text data for Pandas

## Creating random length sentences of random length words using random characters

### Background

A friend recently asked about transforming data in a Pandas data frame and provided an example of what the target column contains currently (prior to the transformation).

After I and a few others made some suggestions, there was also some discussion on performance of the different method of applying the desired transformation.

So that we could properly test, I decided to make some test data, and this in itself provided a challenge.

Briefly, the column of data contained JSON data with:

1. A single-element array
2. Containing an Object (actually, more like a set in Python)
3. Containing one or more words separated by commas

An example (similar to that provided by my friend):

`[{These, words, are, contained, by, an, Object, or, set, which, itself, is, contained, by, an, array}]`  
`[{The, next, set, of, words, is, like, this}]`

As yet, it is not clear whether the data contained the individual words wrapped by double quotes or not, and possibly the data was actually

`[{"These", "words", "are", "contained", "by", "an", "Object", "or", "set", "which", "itself", "is", "contained", "by", "an", "array"}]`  
`[{"The", "next", "set", "of", "words", "is", "like", "this"}]`

Having this example, the challenge is to create a lot of data with a random number of words in each "sentence", where each word contains a random number of letters.

Just the notion of the word "random" implies using `numpy.random` however, none of the methods available in that class can help us directly in our task. For that, we will also need to make use of `itertools` and a not so well known (well, at east to me) method of splitting an array by unequal length intervals using `numpy.split`.

In [1]:
import pandas as pd
from numpy import random as rand
import numpy as np
import itertools as itools
#import unicodedata
import re
import timeit

In [2]:
NUM_RECORDS = 1000000
MAX_WORDS_PER_SENTENCE = 20
MAX_CHARS_PER_WORD = 10

In [3]:
# Create a "template" (of sorts) for our records.
# This template will tell us how many words randomly will be in each record.
sentence_templates = rand.randint(low=1, high=MAX_WORDS_PER_SENTENCE+1, size=NUM_RECORDS)

# Create another "template" which will be a random number of characters for each word.
# We know the count of all words across all records by using the sum() method of the numpy array sentence_templates
word_templates = rand.randint(low=1, high=MAX_CHARS_PER_WORD+1, size=sentence_templates.sum())

In [4]:
# Let's just see how many words we will have in total
len(word_templates)

10498244

In [5]:
# Now use itertools to creating a running count of words
# For example, if the first 5 sentences (records) had word counts of 7, 3, 4, 8, 2
# itertools would return 7, 10, 14, 22, 24
# This running count is needed in the numpy.split method later
words_iter = list(itools.accumulate(word_templates))

In [6]:
# Now let's create an array of random characters from A-Z,
# having length = the last element of our word iterator.
# Similar to the example of 5 above where 24 would be returned as the size.
asc_codes = rand.randint(low=65, high=91, size=words_iter[-1:])

In [7]:
# Let's take a peek at the ascii codes that were generated
asc_codes[:50]

array([85, 78, 66, 87, 67, 73, 71, 84, 77, 65, 67, 71, 74, 81, 65, 82, 87,
       86, 82, 75, 66, 79, 70, 85, 83, 73, 66, 90, 72, 67, 67, 67, 77, 90,
       88, 69, 80, 78, 67, 69, 67, 87, 78, 80, 86, 83, 78, 71, 89, 80])

In [8]:
#f = lambda x: chr(x)
#chars = f(asc_codes.tolist())
# The above did not work, so we will use the following instead
# to convert the ascii codes to characters
chars = [chr(x) for x in asc_codes.tolist()]
chars[:20]

['U',
 'N',
 'B',
 'W',
 'C',
 'I',
 'G',
 'T',
 'M',
 'A',
 'C',
 'G',
 'J',
 'Q',
 'A',
 'R',
 'W',
 'V',
 'R',
 'K']

In [9]:
# Taking the long list of characters that was generated by our conversion from ascii to char
# we use the numpy.split() method in which the 2nd parameter tells us at which index locations
# to split the array (this is why ithertools was handy)
words = [''.join(x) for x in np.split(chars, words_iter[:-1])]

# Let's see the first 20 words we created
words[:20]


['UNBW',
 'CIG',
 'TMAC',
 'GJQARWVR',
 'K',
 'BO',
 'FUSIBZHCCC',
 'MZXEPNCE',
 'C',
 'W',
 'NPV',
 'SNGYPO',
 'KIZZNITZXM',
 'FFXDQYIKVO',
 'BWJO',
 'F',
 'OVPMLWF',
 'CEXM',
 'SKU',
 'YUHI']

In [10]:
# At this point, we have a list of words but we still need to split those words up into sentences (or records)
# Once again, we use itertools to accumulate the count so that we can pass this to the numpy.split() method
sentences_iter = list(itools.accumulate(sentence_templates))

sentences_iter[:20]

[14,
 15,
 30,
 43,
 62,
 70,
 86,
 94,
 99,
 107,
 125,
 136,
 141,
 160,
 173,
 188,
 207,
 212,
 225,
 230]

In [11]:
# Here we apply the numpy.split method, but this time to a list of words rather than a list of characters

# For the source data, if we want the individual words to be wrapped in quotes, use:
#sentences = ['[{"' + str('", "'. join(x)) + '"}]' for x in np.split(words, sentences_iter[:-1])]
# For the source data, if we want no quotes around the individual words (as originally presented by my friend), use:
sentences = ['[{' + str(', '. join(x)) + '}]' for x in np.split(words, sentences_iter[:-1])]
sentences[:10]

['[{UNBW, CIG, TMAC, GJQARWVR, K, BO, FUSIBZHCCC, MZXEPNCE, C, W, NPV, SNGYPO, KIZZNITZXM, FFXDQYIKVO}]',
 '[{BWJO}]',
 '[{F, OVPMLWF, CEXM, SKU, YUHI, GEXMKJWD, HFF, JYQZNP, R, DULZMLAAPV, HJDV, DDAFCED, XMVJTDDR, APAU, F}]',
 '[{XLO, RHKZLL, QWQLHE, EZVVCZ, MP, IVWBTMDVJL, AMLBHZT, QVRS, DDUQJU, NTUGP, FWYGBOV, PFQCF, Y}]',
 '[{UBY, CFIKXQCE, NPZWEGA, BQPZWGBJ, PAIVAZG, GEQZHC, O, VFU, AE, SQHHYL, LUAHATOVVO, PZJCZJITC, BWM, JMUSGIJOD, KGPUAA, S, FXICBFU, IL, N}]',
 '[{OHOVG, XOVPSV, KSHZYWMM, XULZ, TUKJHCI, QYIPPENLMR, MKAM, SDQIKWE}]',
 '[{RS, QVQFDCK, ZDE, F, JZRRVIKVYW, COCMZDG, ATDMF, ISQ, NBLOHJ, YMHZLPS, QCOMJTSNEX, UOKL, MYOB, AAUQUQR, UWEX, X}]',
 '[{YYQDLUUVRO, G, Q, GXUT, JFMF, NJTJ, O, CO}]',
 '[{LSYXUGDL, BO, OXNQYHSMR, NISJXACWY, NONNXGONX}]',
 '[{RYGX, SNJ, LWHCWZID, RTYXJIHU, PQNXGEKJ, CDF, JLYGMM, HOAP}]']

In [12]:
# Now that we have a list of "sentences", we can push them into a Pandas data frame
df_sentences = pd.DataFrame(sentences, columns=["JSON_words"])
df_sentences.head()

,JSON_words
0,"[{UNBW, CIG, TMAC, GJQARWVR, K, BO, FUSIBZHCCC..."
1,[{BWJO}]
2,"[{F, OVPMLWF, CEXM, SKU, YUHI, GEXMKJWD, HFF, ..."
3,"[{XLO, RHKZLL, QWQLHE, EZVVCZ, MP, IVWBTMDVJL,..."
4,"[{UBY, CFIKXQCE, NPZWEGA, BQPZWGBJ, PAIVAZG, G..."


In [13]:
# And now, we can save the data frame so that
# we can repeatedly test against the same data using different methods
df_sentences.to_csv("create_random_words_test_data.csv", sep="|", index=False)

In [14]:
# Read the data back from the file we saved, and re-display the head
df_map = pd.read_csv("create_random_words_test_data.csv", sep="|")
df_map.head()

,JSON_words
0,"[{UNBW, CIG, TMAC, GJQARWVR, K, BO, FUSIBZHCCC..."
1,[{BWJO}]
2,"[{F, OVPMLWF, CEXM, SKU, YUHI, GEXMKJWD, HFF, ..."
3,"[{XLO, RHKZLL, QWQLHE, EZVVCZ, MP, IVWBTMDVJL,..."
4,"[{UBY, CFIKXQCE, NPZWEGA, BQPZWGBJ, PAIVAZG, G..."


In [15]:
%%timeit

# Use numpy.Series.map to transform the data

# If the source data does not contain words wrapped with double quotes, use:
df_map["JSON_words_transformed"] = df_map["JSON_words"].map(lambda x: ['"' + y + '"' for y in re.split("[, ]+", x[2:-2])])
# If the source data already contains words wrapped with double quotes, use:
#df_map["JSON_words"] = [re.sub("[{}]", "", str(x)) for x in df_map["JSON_words"]]

5.29 s ± 29.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
df_map.head()

,JSON_words,JSON_words_transformed
0,"[{UNBW, CIG, TMAC, GJQARWVR, K, BO, FUSIBZHCCC...","[""UNBW"", ""CIG"", ""TMAC"", ""GJQARWVR"", ""K"", ""BO"",..."
1,[{BWJO}],"[""BWJO""]"
2,"[{F, OVPMLWF, CEXM, SKU, YUHI, GEXMKJWD, HFF, ...","[""F"", ""OVPMLWF"", ""CEXM"", ""SKU"", ""YUHI"", ""GEXMK..."
3,"[{XLO, RHKZLL, QWQLHE, EZVVCZ, MP, IVWBTMDVJL,...","[""XLO"", ""RHKZLL"", ""QWQLHE"", ""EZVVCZ"", ""MP"", ""I..."
4,"[{UBY, CFIKXQCE, NPZWEGA, BQPZWGBJ, PAIVAZG, G...","[""UBY"", ""CFIKXQCE"", ""NPZWEGA"", ""BQPZWGBJ"", ""PA..."


In [17]:
df_map.tail()

,JSON_words,JSON_words_transformed
999995,"[{NHUKCSM, DHV, WLMC, DSUC, LOTD, M, BTY, GSQE...","[""NHUKCSM"", ""DHV"", ""WLMC"", ""DSUC"", ""LOTD"", ""M""..."
999996,"[{SDYSTW, DPSRCF, AATRTNQAX, YNHABVAHER, F, HS...","[""SDYSTW"", ""DPSRCF"", ""AATRTNQAX"", ""YNHABVAHER""..."
999997,"[{WCBETOSGM, SWIJJZLFJ}]","[""WCBETOSGM"", ""SWIJJZLFJ""]"
999998,"[{BLAODGAREM, XS, ISENYAWZVF, EKEN, BFER, WSTA...","[""BLAODGAREM"", ""XS"", ""ISENYAWZVF"", ""EKEN"", ""BF..."
999999,"[{JJDISUNY, UQQKEDXKOI, BKMNWW, GNAG, ACOCRMX,...","[""JJDISUNY"", ""UQQKEDXKOI"", ""BKMNWW"", ""GNAG"", ""..."


In [18]:
df_apply = pd.read_csv("create_random_words_test_data.csv", sep="|")
df_apply.head()

,JSON_words
0,"[{UNBW, CIG, TMAC, GJQARWVR, K, BO, FUSIBZHCCC..."
1,[{BWJO}]
2,"[{F, OVPMLWF, CEXM, SKU, YUHI, GEXMKJWD, HFF, ..."
3,"[{XLO, RHKZLL, QWQLHE, EZVVCZ, MP, IVWBTMDVJL,..."
4,"[{UBY, CFIKXQCE, NPZWEGA, BQPZWGBJ, PAIVAZG, G..."


In [19]:
%%timeit

# Use numpy.Series.apply to transform the data

df_apply["JSON_words_transformed"] = df_apply["JSON_words"].apply(lambda x: ['"' + y + '"' for y in re.split("[, ]+", x[2:-2])])

5.31 s ± 46.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
df_apply.head()

,JSON_words,JSON_words_transformed
0,"[{UNBW, CIG, TMAC, GJQARWVR, K, BO, FUSIBZHCCC...","[""UNBW"", ""CIG"", ""TMAC"", ""GJQARWVR"", ""K"", ""BO"",..."
1,[{BWJO}],"[""BWJO""]"
2,"[{F, OVPMLWF, CEXM, SKU, YUHI, GEXMKJWD, HFF, ...","[""F"", ""OVPMLWF"", ""CEXM"", ""SKU"", ""YUHI"", ""GEXMK..."
3,"[{XLO, RHKZLL, QWQLHE, EZVVCZ, MP, IVWBTMDVJL,...","[""XLO"", ""RHKZLL"", ""QWQLHE"", ""EZVVCZ"", ""MP"", ""I..."
4,"[{UBY, CFIKXQCE, NPZWEGA, BQPZWGBJ, PAIVAZG, G...","[""UBY"", ""CFIKXQCE"", ""NPZWEGA"", ""BQPZWGBJ"", ""PA..."


In [21]:
df_apply.tail()

,JSON_words,JSON_words_transformed
999995,"[{NHUKCSM, DHV, WLMC, DSUC, LOTD, M, BTY, GSQE...","[""NHUKCSM"", ""DHV"", ""WLMC"", ""DSUC"", ""LOTD"", ""M""..."
999996,"[{SDYSTW, DPSRCF, AATRTNQAX, YNHABVAHER, F, HS...","[""SDYSTW"", ""DPSRCF"", ""AATRTNQAX"", ""YNHABVAHER""..."
999997,"[{WCBETOSGM, SWIJJZLFJ}]","[""WCBETOSGM"", ""SWIJJZLFJ""]"
999998,"[{BLAODGAREM, XS, ISENYAWZVF, EKEN, BFER, WSTA...","[""BLAODGAREM"", ""XS"", ""ISENYAWZVF"", ""EKEN"", ""BF..."
999999,"[{JJDISUNY, UQQKEDXKOI, BKMNWW, GNAG, ACOCRMX,...","[""JJDISUNY"", ""UQQKEDXKOI"", ""BKMNWW"", ""GNAG"", ""..."


In [22]:
# What about using str.replace or re.sub?

df_replace = pd.read_csv("create_random_words_test_data.csv", sep="|")
df_replace.head()

,JSON_words
0,"[{UNBW, CIG, TMAC, GJQARWVR, K, BO, FUSIBZHCCC..."
1,[{BWJO}]
2,"[{F, OVPMLWF, CEXM, SKU, YUHI, GEXMKJWD, HFF, ..."
3,"[{XLO, RHKZLL, QWQLHE, EZVVCZ, MP, IVWBTMDVJL,..."
4,"[{UBY, CFIKXQCE, NPZWEGA, BQPZWGBJ, PAIVAZG, G..."


In [23]:
%%timeit

# If we are certain that the data is in a specific format for which we know
# we can perform regex substitutions, or string replacements, we may get better performance

df_replace["JSON_words_transformed"] = [str(x)
                                        .replace(', ', '", "')
                                        .replace('[{', '["')
                                        .replace('}]', '"]')
                                        for x in df_replace["JSON_words"]]

1.13 s ± 4.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
df_replace.head()

,JSON_words,JSON_words_transformed
0,"[{UNBW, CIG, TMAC, GJQARWVR, K, BO, FUSIBZHCCC...","[""UNBW"", ""CIG"", ""TMAC"", ""GJQARWVR"", ""K"", ""BO"",..."
1,[{BWJO}],"[""BWJO""]"
2,"[{F, OVPMLWF, CEXM, SKU, YUHI, GEXMKJWD, HFF, ...","[""F"", ""OVPMLWF"", ""CEXM"", ""SKU"", ""YUHI"", ""GEXMK..."
3,"[{XLO, RHKZLL, QWQLHE, EZVVCZ, MP, IVWBTMDVJL,...","[""XLO"", ""RHKZLL"", ""QWQLHE"", ""EZVVCZ"", ""MP"", ""I..."
4,"[{UBY, CFIKXQCE, NPZWEGA, BQPZWGBJ, PAIVAZG, G...","[""UBY"", ""CFIKXQCE"", ""NPZWEGA"", ""BQPZWGBJ"", ""PA..."


In [25]:
df_replace.tail()

,JSON_words,JSON_words_transformed
999995,"[{NHUKCSM, DHV, WLMC, DSUC, LOTD, M, BTY, GSQE...","[""NHUKCSM"", ""DHV"", ""WLMC"", ""DSUC"", ""LOTD"", ""M""..."
999996,"[{SDYSTW, DPSRCF, AATRTNQAX, YNHABVAHER, F, HS...","[""SDYSTW"", ""DPSRCF"", ""AATRTNQAX"", ""YNHABVAHER""..."
999997,"[{WCBETOSGM, SWIJJZLFJ}]","[""WCBETOSGM"", ""SWIJJZLFJ""]"
999998,"[{BLAODGAREM, XS, ISENYAWZVF, EKEN, BFER, WSTA...","[""BLAODGAREM"", ""XS"", ""ISENYAWZVF"", ""EKEN"", ""BF..."
999999,"[{JJDISUNY, UQQKEDXKOI, BKMNWW, GNAG, ACOCRMX,...","[""JJDISUNY"", ""UQQKEDXKOI"", ""BKMNWW"", ""GNAG"", ""..."
